##PPGC - UFPEL
##2024/1 - 1110076 - 1 - TÓPICOS ESPECIAIS EM COMPUTAÇÃO IV - MINERAÇÃO DE DADOS EDUCACIONAIS

##Guilherme D. Lima - Mestrando em Computação

##O que é MDE e qual seu objetivo?

###A Mineração de Dados Educacionais (MDE) é uma área interdisciplinar que combina técnicas de mineração de dados, aprendizado de máquina, estatística e outras disciplinas relacionadas para analisar grandes conjuntos de dados educacionais. Seu principal objetivo é extrair conhecimento útil e insights a partir desses dados, visando melhorar a eficácia do ensino e da aprendizagem. A MDE busca entender padrões de comportamento dos alunos, identificar fatores que influenciam o desempenho acadêmico, personalizar o ensino de acordo com as necessidades individuais dos alunos, prever resultados futuros e apoiar a tomada de decisões educacionais baseadas em evidências. Ao aplicar técnicas de análise de dados ao contexto educacional, a MDE busca promover aprimoramentos significativos na qualidade da educação e no sucesso dos alunos.

##Atividade realizada em aula 1

###Descrição:
✓ Uma rede de escolas privadas tem uma base de dados com vários dados
sociodemográficos e de notas dos estudantes em matemática no oitavo ano do ensino
fundamental.

✓ Nessa escola, os alunos realizam duas provas importantes de matemática, que são
responsáveis pela nova final em matemática:

✓ prova1: prova em julho, no meio do ano letivo;

✓ prova2: prova em dezembro, no final do ano letivo.

✓ A nota final na disciplina é dada pela média da Prova-1 e Prova-2.

✓ A escola gostaria de verificar se é possível determinar na metade do ano letivo, após a
realização da 1ª prova, se o aluno será aprovado no final do ano ou não, com base nas
informações sociodemográficas e de notas que tem até o momento. Os gestores da
escola pretendem usar essa informação para oferecer apoio pedagógico aos alunos
em risco de ser reprovado.

✓ Os dados podem ser obtidos do sistema computacional de acompanhamento de
notas, através de uma opção de exportação. Essa opção gera um arquivo csv.

###1. Definição do Escopo a ser trabalhado

####✓ Determinar se um aluno será aprovado em matemática com base nas notas da prova-1 e dos dados sociodemográficos

####✓ Problema de classificação binária:
#####✓ aluno foi aprovado ou não

###Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as skl
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

###2.1 e 2.2 Carregamento dos Dados

In [ ]:
variable = 'aula2-exemplo.csv'
dataSet = pd.read_csv(variable)
dataSet.head()

,Unnamed: 0,Matricula,Idade,Sexo,Prova1,Prova2,HorasEstudo
0,0,1,24,F,8.9,8.2,1.7
1,1,2,37,NF,9.9,2.1,5.4
2,2,3,46,M,NaN,3.8,1.1
3,3,4,32,M,8.8,9.7,0.0
4,4,5,28,M,5.2,3.4,5.5


###Pré-Processamento dos Dados

In [ ]:
dataSet = dataSet.dropna() # Remove as linhas onde há valores nulos

dataSet = dataSet.drop_duplicates() # Remove os valores duplicados

dataSet.head()

,Unnamed: 0,Matricula,Idade,Sexo,Prova1,Prova2,HorasEstudo
0,0,1,24,F,8.9,8.2,1.7
1,1,2,37,NF,9.9,2.1,5.4
3,3,4,32,M,8.8,9.7,0.0
4,4,5,28,M,5.2,3.4,5.5
6,6,7,46,M,5.7,9.2,13.4


In [ ]:
dataModify = dataSet[['Idade', 'Sexo', 'Prova1', 'HorasEstudo']] # Separando o dataset inicial selecionando apenas as colunas de interesse

dataModify.head()

,Idade,Sexo,Prova1,HorasEstudo
0,24,F,8.9,1.7
1,37,NF,9.9,5.4
3,32,M,8.8,0.0
4,28,M,5.2,5.5
6,46,M,5.7,13.4


In [ ]:
# Criar dummies apenas para as colunas 'Sexo'
sexo_dummies = pd.get_dummies(dataModify['Sexo'], prefix='Sexo')

# Converter para inteiros apenas as colunas específicas
colunas_a_converter = ['Sexo_F', 'Sexo_M', 'Sexo_NF']
dataModify[colunas_a_converter] = sexo_dummies[colunas_a_converter].astype(int) # Normalizando para 0 onde é False e 1 onde é True
dataModify = dataModify.drop(columns=['Sexo'])
dataModify.head()

<ipython-input-7-691749516fda>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataModify[colunas_a_converter] = sexo_dummies[colunas_a_converter].astype(int) # Normalizando para 0 onde é False e 1 onde é True
<ipython-input-7-691749516fda>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataModify[colunas_a_converter] = sexo_dummies[colunas_a_converter].astype(int) # Normalizando para 0 onde é False e 1 onde é True


,Idade,Prova1,HorasEstudo,Sexo_F,Sexo_M,Sexo_NF
0,24,8.9,1.7,1,0,0
1,37,9.9,5.4,0,0,1
3,32,8.8,0.0,0,1,0
4,28,5.2,5.5,0,1,0
6,46,5.7,13.4,0,1,0


In [ ]:
df_media = pd.DataFrame({'media': (dataSet['Prova1'] + dataSet['Prova2'])/2}) # Calculando a média entre as duas provas

df_media.head()

,media
0,8.55
1,6.00
3,9.25
4,4.30
6,7.45


In [ ]:
df_aprovados = pd.DataFrame({'Aprovado': [1 if media>=6 else 0 for media in df_media['media']]}) # 0 para reprovado e 1 para aprovado, para aprovação deve ter uma média >= 6
df_aprovados.head()

,Aprovado
0,1
1,1
2,1
3,0
4,1


In [ ]:
y = df_aprovados.values

y = y.flatten()

x = dataModify

###Treinamento do Modelo

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=35) #Separando os dados para o modelo

In [ ]:
modelo = LogisticRegression()
modelo.fit(X_train, Y_train)

LogisticRegression()

###Avaliando o Modelo

In [ ]:
y_pred = modelo.predict(X_test)
accuracy = accuracy_score(Y_test, y_pred)
precision = precision_score(Y_test, y_pred)
recall = recall_score(Y_test, y_pred)
f1 = f1_score(Y_test, y_pred)

In [ ]:
print(f'Acuracia: {accuracy}') # Quantos aprovados e reprovados preditos corretamente
print(f'Precision: {precision}') # Número Total de preditos como aprovados, quantos são mesmo aprovados
print(f'recall: {recall}') # Número total de aprovados na base de dados qauntos foram preditos como aprovados
print(f'f1: {f1}') # Média harmônica entre Precisão e Recall

Acuracia: 0.8066666666666666
Precision: 0.6831683168316832
recall: 0.7263157894736842
f1: 0.7040816326530612
